In [123]:
import pandas
!pip install nltk
import nltk
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

# data_path = Path("/mnt/f/workspace/DataCLUE/nips_expr/aclImdb")
data_path = Path("/home/jovyan/Data Mining Grad Project/IMDB")

nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = stopwords.words('english')

# stopwords = set(Path("/mnt/f/workspace/DataCLUE/nips_expr/stopwords.txt").read_text().split("\n"))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
from tqdm import tqdm
def load_data(path):
    labels = []
    texts = []
    for file_path in tqdm((path / "neg").glob("*.txt")):
        labels.append(0)
        texts.append(file_path.read_text())
    
    for file_path in tqdm((path / "pos").glob("*.txt")):
        labels.append(1)
        texts.append(file_path.read_text())

    return texts, labels

all_train_texts, train_Y = load_data(data_path / "train")
all_test_texts, test_Y = load_data(data_path / "test")

1000it [00:00, 1368.07it/s]
1000it [00:00, 1393.14it/s]
1000it [00:00, 1370.13it/s]
1000it [00:00, 1344.75it/s]


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
def data_quality(vectorizer, X, train_Y, cut_prop=1, filter_set = set()):
    from sklearn.feature_selection import f_classif, chi2
    F, pvalues_f = chi2(X, train_Y)
    
    last = int(len(F) * cut_prop)
    sorted_F = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)[:last]
    
    values = [value for name, value in sorted_F if name not in filter_set]
    # max_value, min_value = max(values), min(values)



    # values = [(value - min_value) / (max_value - min_value) for value in values]

    # print(values[:10])
    # print(values[-10:])

    return sum(values) / len(values)

In [43]:
def naive_bayes_classifier(train_x, train_y, test_x):  
    from sklearn.naive_bayes import MultinomialNB  
    model = MultinomialNB(alpha=0.01)  
    model.fit(train_x, train_y)
    return model.predict(test_x)

def svm_classifier(train_x, train_y, test_x):
    from sklearn.svm import SVC  
    model = SVC(kernel='rbf', probability=True, max_iter=50)
    model.fit(train_x, train_y)
    return model.predict(test_x)

def lgb_classifier(train_x, train_y, test_x):
    !pip install lightgbm
    import lightgbm as lgb
    import numpy as np
    
    train_data = lgb.Dataset(train_x.astype('float32'), label=np.array(train_y, np.float32))
#     test_data = lgb.Dataset(test_x, label=test_y)
    
    params={
#         'learning_rate':0.1,
#         'lambda_l1':0.1,
#         'lambda_l2':0.2,
#         'max_depth':6,
#         'min_data_'
        'objective':'multiclass',
        'num_iteration': 200, 
        'num_class': 2,
    }
    
    model = lgb.train(params, train_data)
    return np.argmax(model.predict(test_x.astype('float32')), axis=1)

def fasttext_classifier(train_texts, train_y, test_texts, test_y, name):
    !pip install fasttext
    import fasttext

    model_path = data_path / f"fasttext.model.{name}.bin"
    if model_path.exists():
        classifier = fasttext.load_model(str(model_path))
    else:
        write_path = data_path / f"fasttext.data.train.{name}.txt"
        write_path.write_text("\n".join(
            f"{sentence}\t__label__{label}" for sentence, label in zip(train_texts, train_Y)
        ))

        classifier = fasttext.train_supervised(str(write_path), lr=0.1, word_ngrams=3, bucket=2000000, label_prefix="__label__")
        classifier.save_model(str(model_path))
        
    labels, _ = classifier.predict(test_texts)
    y_pred = [int(label[0][-1]) for label in labels]
    return y_pred

def knn_classifier(train_X, train_Y, test_X):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier(n_neighbors=3)  
    model.fit(train_X, train_Y)
    return model.predict(test_X)

def textcnn(train_x, train_y):
    from deepclassifier.models import TextCNN
    from deepclassifier.trainers import Trainer
    import torch.optim as optim
    
    model = TextCNN(embedding_dim=300, dropout_rate=0.2, num_class=2)
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()


In [44]:
model_mapper = {
    "nb": naive_bayes_classifier,
    "svm": svm_classifier,
    "lgb": lgb_classifier,
    "knn": knn_classifier,
    "fasttext": None,
    "textcnn": None,
}

def train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, model_name):
    if model_name not in model_mapper:
        y_pred = fasttext_classifier(train_texts, train_Y, test_texts, test_Y, model_name)
    else:
        y_pred = model_mapper[model_name](train_X, train_Y, test_X)
    from sklearn.metrics import accuracy_score
    return accuracy_score(test_Y, y_pred)

# No Operations_Normal

In [45]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)


from sklearn.feature_selection import f_classif, chi2
F, pvalues_f = chi2(train_X, train_Y)

sorted_F = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)

In [46]:
all_train_texts[0]

'and rent a GOOD horror movie. It\'s like the writer had never seen a horror movie before and didn\'t realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like "Scream" and "Scary Movie".<br /><br />In between the scary bits is the most BANAL and BORING dialog ever written. Stupid "we\'re going to the prom" junk. I wanted to claw my ears off. Honestly, "The Hills" has better dialog.<br /><br />There really was no need to make this movie. Leading lady is uninteresting and I kept thinking "Her? Really? Guy is obsessed with her? Really?" <br /><br />All the characters act in stupid ways, including the police. (Cover the place in teams of 2! Front and back! Not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed.) <br /><br />The serial killer just swans about murdering everyone he wants without the least bit of problem. No resistance from victims (or doors). Nobody has any protection

In [47]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

data_quality(vectorizer, train_X, train_Y)

1.9660682259796485

In [48]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "nb")

0.7385

In [49]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "svm")

0.529

In [50]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.107734 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7675
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1948
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.8055

In [51]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "normal")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=308257 sha256=4a6db45150eab5b19b8087f85995b2857afbf1beb5c482e4138fdb3b4dcb4491
  Stored in directory: /home/jovyan/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


Read 0M words
Number of words:  52283
Number of labels: 2
Progress: 100.0% words/sec/thread:  187711 lr:  0.000000 avg.loss:  0.694590 ETA:   0h 0m 0s100.0% words/sec/thread:  187713 lr: -0.000034 avg.loss:  0.694590 ETA:   0h 0m 0s


0.5175

In [52]:
train_model(all_train_texts, train_X, train_Y, all_test_texts, test_X, test_Y, "knn")

0.5535

# LowerCase

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

train_texts = [sentence.lower() for sentence in all_train_texts]
test_texts = [sentence.lower() for sentence in all_test_texts]

vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

2.095419358521762

In [57]:
train_texts[0]

'and rent a good horror movie. it\'s like the writer had never seen a horror movie before and didn\'t realize every single thing he wrote was clichéd and hackneyed and has been parodied to perfection in movies like "scream" and "scary movie".<br /><br />in between the scary bits is the most banal and boring dialog ever written. stupid "we\'re going to the prom" junk. i wanted to claw my ears off. honestly, "the hills" has better dialog.<br /><br />there really was no need to make this movie. leading lady is uninteresting and i kept thinking "her? really? guy is obsessed with her? really?" <br /><br />all the characters act in stupid ways, including the police. (cover the place in teams of 2! front and back! not one sleepy cop sitting in his car with the window rolled down just waiting for his throat to be slashed.) <br /><br />the serial killer just swans about murdering everyone he wants without the least bit of problem. no resistance from victims (or doors). nobody has any protection

In [58]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.7355

In [59]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5175

In [60]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.102661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7658
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1904
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.8115

In [61]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lower")

Read 0M words
Number of words:  48062
Number of labels: 2
Progress: 100.0% words/sec/thread:  197380 lr:  0.000000 avg.loss:  0.695870 ETA:   0h 0m 0s


0.6395

In [62]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.5605

# Stem

In [64]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('punkt')

porter = PorterStemmer()

def stem_sentences(sentences: str) -> str:
    def stem_sentence(sentence):
        words = nltk.word_tokenize(sentence)
        return " ".join(
            porter.stem(word, to_lowercase=False) for word in words
        )
    return [stem_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = stem_sentences(all_train_texts), stem_sentences(all_test_texts)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

100%|██████████| 2000/2000 [00:13<00:00, 144.81it/s]

100%|██████████| 2000/2000 [00:13<00:00, 146.08it/s]


In [65]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

2.3851499590692637

In [66]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.7285

In [67]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5125

In [68]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.110489 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7649
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1824
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.8065

In [69]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "stem")

Read 0M words
Number of words:  23721
Number of labels: 2
Progress: 100.0% words/sec/thread:  203069 lr:  0.000000 avg.loss:  0.694611 ETA:   0h 0m 0s


0.5005

In [70]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.5695

# Lemmatization

In [77]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [78]:
def lemma_sentences(sentences: str) -> str:
    def lemma_sentence(sentence):
        return " ".join(
            wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(sentence)
        )
    return [lemma_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = lemma_sentences(all_train_texts), lemma_sentences(all_test_texts)





  0%|          | 0/2000 [00:12<?, ?it/s]




  0%|          | 1/2000 [00:01<57:08,  1.72s/it]



  1%|▏         | 29/2000 [00:01<01:29, 21.96it/s]



  3%|▎         | 58/2000 [00:01<00:40, 48.20it/s]



  4%|▍         | 86/2000 [00:02<00:25, 76.24it/s]



  6%|▌         | 111/2000 [00:02<00:18, 101.99it/s]



  7%|▋         | 140/2000 [00:02<00:13, 134.86it/s]



  9%|▉         | 179/2000 [00:02<00:09, 185.68it/s]



 11%|█         | 216/2000 [00:02<00:07, 226.06it/s]



 12%|█▏        | 249/2000 [00:02<00:07, 244.94it/s]



 14%|█▍        | 281/2000 [00:02<00:06, 254.78it/s]



 16%|█▌        | 317/2000 [00:02<00:05, 280.63it/s]



 17%|█▋        | 349/2000 [00:02<00:05, 283.94it/s]



 19%|█▉        | 381/2000 [00:02<00:05, 272.61it/s]



 21%|██        | 411/2000 [00:03<00:05, 276.54it/s]



 22%|██▏       | 444/2000 [00:03<00:05, 290.97it/s]



 24%|██▍       | 475/2000 [00:03<00:05, 275.51it/s]



 25%|██▌       | 504/2000 [00:03<00:05, 278.80it/s]



 27%|██▋       | 534/2000

In [79]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

2.162227993528516

In [80]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.73

In [81]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5285

In [82]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.096782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7585
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1861
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.812

In [83]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lemma")

Read 0M words
Number of words:  29056
Number of labels: 2
Progress: 100.0% words/sec/thread:  207516 lr:  0.000000 avg.loss:  0.695225 ETA:   0h 0m 0s


0.508

In [84]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.5595

# Remove Stop

In [87]:
remove_stop_sentences([all_train_texts[0]])

100%|██████████| 1/1 [00:00<00:00, 156.71it/s]


["rent GOOD horror movie . It 's like writer never seen horror movie n't realize every single thing wrote clichéd hackneyed parodied perfection movies like `` Scream '' `` Scary Movie '' . < br / > < br / > In scary bits BANAL BORING dialog ever written . Stupid `` 're going prom '' junk . I wanted claw ears . Honestly , `` The Hills '' better dialog. < br / > < br / > There really need make movie . Leading lady uninteresting I kept thinking `` Her ? Really ? Guy obsessed ? Really ? '' < br / > < br / > All characters act stupid ways , including police . ( Cover place teams 2 ! Front back ! Not one sleepy cop sitting car window rolled waiting throat slashed . ) < br / > < br / > The serial killer swans murdering everyone wants without least bit problem . No resistance victims ( doors ) . Nobody protection least idea fighting back ( flipping security lock hotel room door ) . The people like mentally disabled sheep. < br / > < br / > By , 're gore fan , 'll disappointed . All killing kep

In [88]:
def remove_stop_sentences(sentences: str) -> str:
    def remove_stop_sentence(sentence):
        return " ".join(
            word for word in nltk.word_tokenize(sentence) if word not in stopWords
        )
    return [remove_stop_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = remove_stop_sentences(all_train_texts), remove_stop_sentences(all_test_texts)

100%|██████████| 2000/2000 [00:05<00:00, 397.28it/s]


In [89]:
vectorizer = CountVectorizer(lowercase=True)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

2.04981558911457

In [90]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.726

In [91]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5075

In [92]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.181268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6858
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1860
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.8135

In [93]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "rem_stop")

Read 0M words
Number of words:  31113
Number of labels: 2
Progress: 100.0% words/sec/thread:  205149 lr:  0.000000 avg.loss:  0.694947 ETA:   0h 0m 0s


0.5475

In [94]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.5375

# f_classify threshold

In [95]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

from sklearn.feature_selection import f_classif
F, pvalues_f = f_classif(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.2):]]

def remove_words_sentences(sentences: str, removed_words: set) -> str:
    def remove_words_sentence(sentence):
        return " ".join(
            word for word in nltk.word_tokenize(sentence) if word not in removed_words
        )
    return [remove_words_sentence(sentence) for sentence in tqdm(sentences)]

train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))


vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

100%|██████████| 2000/2000 [00:04<00:00, 465.35it/s]


2.3417035155926635

In [96]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.7305

In [97]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5225

In [98]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.103318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5690
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1419
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.81

In [99]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "remove_f_thre")

Read 0M words
Number of words:  25894
Number of labels: 2
Progress: 100.0% words/sec/thread:  221908 lr:  0.000000 avg.loss:  0.694734 ETA:   0h 0m 0s


0.5

In [100]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.554

# CHI2_threshold

In [101]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)

from sklearn.feature_selection import chi2
F, pvalues_f = chi2(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.4):]]


In [102]:
remove_words_sentences([all_train_texts[0]], set(filter_names))

100%|██████████| 1/1 [00:00<00:00, 56.15it/s]


["and rent a GOOD horror movie . 's like the had a horror movie and n't realize single thing he wrote was clichéd and hackneyed and has been to perfection in like `` '' and `` Scary Movie '' . < br / > < br / > between the scary bits is the most BANAL and BORING dialog ever . Stupid `` 're going to the prom '' junk . I to claw my off . Honestly , `` Hills '' has better dialog. < br / > < br / > There really was no to make this movie . is uninteresting and I thinking `` ? Really ? Guy is obsessed with her ? Really ? '' < br / > < br / > the act in stupid ways , the police . ( Cover the in of 2 ! and ! Not one sleepy cop sitting in his with the just for his throat to be slashed . ) < br / > < br / > serial killer just about he without the least bit of problem . No from victims ( or doors ) . Nobody has any or the least idea of ( or the security on the hotel ) . like mentally sheep. < br / > < br / > the , if 're a gore fan , 'll be disappointed too . the killing is and is -- -- . ( So bo

In [103]:
train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))

vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)

data_quality(vectorizer, train_X, train_Y)

100%|██████████| 2000/2000 [00:04<00:00, 467.67it/s]


2.719204270388087

In [104]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.74

In [105]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.5265

In [106]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.104744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5328
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1302
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.804

In [107]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "chi2_04")

Read 0M words
Number of words:  20772
Number of labels: 2
Progress: 100.0% words/sec/thread:  220663 lr:  0.000000 avg.loss:  0.695797 ETA:   0h 0m 0s100.0% words/sec/thread:  220665 lr: -0.000020 avg.loss:  0.695797 ETA:   0h 0m 0s


0.5665

In [108]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.555

# CHI2_02

In [109]:
vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(all_train_texts)
test_X = vectorizer.transform(all_test_texts)

from sklearn.feature_selection import chi2
F, pvalues_f = chi2(train_X, train_Y)
value_names = sorted(zip(vectorizer.get_feature_names(), F), key=lambda x: x[1], reverse=True)
filter_names = [name for name, value in value_names[-int(len(value_names)*0.2):]]

data_quality(vectorizer, train_X, train_Y, 1, set(filter_names))

2.4142181209029294

In [110]:
train_texts, test_texts = remove_words_sentences(all_train_texts, set(filter_names)), remove_words_sentences(all_test_texts, set(filter_names))

vectorizer = CountVectorizer(lowercase=False)
train_X = vectorizer.fit_transform(train_texts)
test_X = vectorizer.transform(test_texts)
data_quality(vectorizer, train_X, train_Y)

100%|██████████| 2000/2000 [00:04<00:00, 464.68it/s]


2.3431034490585363

In [111]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "nb")

0.732

In [112]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "svm")

0.524

In [113]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "lgb")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100478 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5712
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 1414
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -0.693147


0.8055

In [114]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "chi2_02")

Read 0M words
Number of words:  25893
Number of labels: 2
Progress: 100.0% words/sec/thread:  218403 lr:  0.000000 avg.loss:  0.693890 ETA:   0h 0m 0s


0.501

In [115]:
train_model(train_texts, train_X, train_Y, test_texts, test_X, test_Y, "knn")

0.5625

In [116]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)  
neigh.fit(train_X, train_Y)

KNeighborsClassifier(n_neighbors=3)

In [120]:
result = neigh.predict(test_X)